# 통합 Inference: 보트 유형 분류, 승선 인원수 탐지, 번호판 OCR

## 개요
이 노트북은 수상 레저 이미지에 대해 다음 세 가지 inference를 수행합니다:

### 연속 실행 과업 (순차적)
1. **보트 유형 분류**: 학습된 보트 유형 분류 모델로 보트를 탐지하고 유형을 분류 (모터보트, 수상오토바이, 고무보트, 세일링요트, 기타)
2. **승선 인원수 탐지**: 보트 유형 분류에서 나온 보트 좌표와 YOLOv11s 모델로 탐지한 사람 좌표를 함께 사용하여 각 보트의 탑승 인원수를 계산

### 독립 실행 과업
3. **번호판 OCR**: 학습된 번호판 탐지 모델과 EasyOCR을 사용하여 번호판 텍스트 추출 (보트 유형 분류 및 승선 인원수 탐지와 독립적으로 실행)

## 실행 순서
1. **보트 유형 분류** → `result/boat_classification/` (보트 바운딩 박스 + 유형 라벨)
2. **승선 인원수 탐지** → `result/boat_classification/` (보트 유형 분류 결과에 사람 바운딩 박스 + 탑승 인원 수 추가하여 덮어쓰기)
3. **번호판 OCR** → `result/plate_ocr/` (별도로 독립 실행)

## 결과 저장
- `result/boat_classification/`: 보트 유형 분류 및 승선 인원수 탐지 통합 결과
  - `classification_results.csv`: 보트 분류 결과 CSV 파일
  - `*_result.jpg`: 보트 유형 + 탑승 인원 수가 표시된 최종 시각화 이미지
- `result/plate_ocr/`: 번호판 OCR 결과
  - `overlay/`: 탐지 결과 오버레이 이미지
  - `crops/`: 탐지된 번호판 크롭 이미지
  - `ocr_results.csv`: OCR 결과 CSV 파일
  - `run_log.txt`: 실행 로그

## 입력 데이터
- `data/test/` 폴더의 테스트 이미지 파일들
- 승선 인원수 탐지는 보트 유형 분류 결과 이미지(`result/boat_classification/*_result.jpg`)를 사용하여 처리하고, 결과를 같은 폴더에 저장


In [ ]:
# 필요한 라이브러리 import
import sys
from pathlib import Path
import torch
import os
import glob
import cv2
import numpy as np
from ultralytics import YOLO

# 프로젝트 루트 경로 설정
BASE_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(BASE_DIR / "src"))

# 경로를 상대 경로로 변환하는 함수 (개인 경로 노출 방지)
def get_relative_path(path):
    """절대 경로를 프로젝트 루트 기준 상대 경로로 변환"""
    try:
        path = Path(path)
        if path.is_absolute():
            try:
                rel_path = path.relative_to(BASE_DIR)
                # 프로젝트 루트 자체는 "."로 표시
                return "." if rel_path == Path(".") else rel_path
            except ValueError:
                # BASE_DIR 밖의 경로는 그대로 반환 (하지만 사용자명은 마스킹)
                return str(path).replace(os.path.expanduser("~"), "~")
        return path
    except:
        return str(path)

print(f"프로젝트 루트: .")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## 1. 설정 및 경로 확인


In [ ]:
# 경로 설정
PASSENGER_MODEL_PATH = BASE_DIR / "model" / "yolo11s_passenger_counting.pt"
PLATE_MODEL_PATH = BASE_DIR / "model" / "plate_detection_baseline.pt"
BOAT_CLASSIFICATION_MODEL_PATH = BASE_DIR / "model" / "boat_classification_baseline.pt"
TEST_PATH = BASE_DIR / "data" / "test"

# 결과 저장 경로 (통일된 result 폴더)
RESULT_DIR = BASE_DIR / "result"
PLATE_RESULT_DIR = RESULT_DIR / "plate_ocr"
BOAT_CLASSIFICATION_RESULT_DIR = RESULT_DIR / "boat_classification"
# 승선 인원수 탐지 결과는 보트 유형 분류 결과 폴더에 저장

print(f"[설정] 승선 인원수 모델: {get_relative_path(PASSENGER_MODEL_PATH)}")
print(f"[설정] 번호판 탐지 모델: {get_relative_path(PLATE_MODEL_PATH)}")
print(f"[설정] 보트 유형 분류 모델: {get_relative_path(BOAT_CLASSIFICATION_MODEL_PATH)}")
print(f"[설정] 테스트 이미지 경로: {get_relative_path(TEST_PATH)}")
print(f"[설정] 결과 저장 경로: {get_relative_path(RESULT_DIR)}")
print()

# 파일 존재 확인
if not PASSENGER_MODEL_PATH.exists():
    print(f"[ERROR] 승선 인원수 모델 파일이 없습니다: {get_relative_path(PASSENGER_MODEL_PATH)}")
else:
    print(f"[OK] 승선 인원수 모델 확인: {get_relative_path(PASSENGER_MODEL_PATH)}")

if not PLATE_MODEL_PATH.exists():
    print(f"[ERROR] 번호판 탐지 모델 파일이 없습니다: {get_relative_path(PLATE_MODEL_PATH)}")
else:
    print(f"[OK] 번호판 탐지 모델 확인: {get_relative_path(PLATE_MODEL_PATH)}")

if not BOAT_CLASSIFICATION_MODEL_PATH.exists():
    print(f"[ERROR] 보트 유형 분류 모델 파일이 없습니다: {get_relative_path(BOAT_CLASSIFICATION_MODEL_PATH)}")
else:
    print(f"[OK] 보트 유형 분류 모델 확인: {get_relative_path(BOAT_CLASSIFICATION_MODEL_PATH)}")

if not TEST_PATH.exists():
    print(f"[ERROR] 테스트 이미지 폴더가 없습니다: {get_relative_path(TEST_PATH)}")
else:
    image_count = len(list(TEST_PATH.glob("*.jpg"))) + len(list(TEST_PATH.glob("*.png")))
    print(f"[OK] 테스트 이미지 폴더 확인: {get_relative_path(TEST_PATH)} ({image_count}개 이미지)")

# 결과 폴더 생성
RESULT_DIR.mkdir(exist_ok=True)
PLATE_RESULT_DIR.mkdir(exist_ok=True)
BOAT_CLASSIFICATION_RESULT_DIR.mkdir(exist_ok=True)


## 2. 보트 유형 분류 (Boat Classification)


In [ ]:
# =============================================================
# 보트 유형 분류 실행 (src/boat_classification_inference.py 사용)
# =============================================================

print("=" * 60)
print("보트 유형 분류 시작")
print("=" * 60)

# boat_classification_inference.py 실행
import subprocess
result = subprocess.run(
    [sys.executable, str(BASE_DIR / "src" / "boat_classification_inference.py")],
    cwd=str(BASE_DIR),
    capture_output=True,
    text=True,
    encoding='utf-8',
    errors='replace'
)

print(result.stdout)
if result.stderr:
    print("[WARN] stderr:", result.stderr)

BOAT_CSV_PATH = BOAT_CLASSIFICATION_RESULT_DIR / "classification_results.csv"
print(f"\n✅ 보트 유형 분류 완료!")
print(f"[DONE] 결과 CSV: {get_relative_path(BOAT_CSV_PATH)}")
print(f"[DONE] 결과 이미지: {get_relative_path(BOAT_CLASSIFICATION_RESULT_DIR)}")




In [ ]:
# =============================================================
# 승선 인원수 탐지 함수들
# =============================================================

# 장치 설정
try:
    if torch.cuda.is_available():
        test_tensor = torch.zeros(1, device="cuda:0")
        result = test_tensor + 1
        gpu_name = torch.cuda.get_device_name(0)
        device = 0
        print(f"[INFO] 사용 장치: GPU ({gpu_name})")
    else:
        device = "cpu"
        print(f"[INFO] 사용 장치: CPU")
except (RuntimeError, Exception) as e:
    print(f"[WARN] GPU 사용 불가 (오류: {str(e)[:100]}), CPU로 전환합니다.")
    device = "cpu"
    print(f"[INFO] 사용 장치: CPU")

# 모델 로딩
passenger_model = YOLO(str(PASSENGER_MODEL_PATH))

# IoU 계산
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x2 <= x1 or y2 <= y1:
        return 0.0
    inter = (x2 - x1) * (y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter / (area1 + area2 - inter + 1e-6)

# 인접 박스 병합
def merge_close_boxes(boxes, iou_thresh=0.6):
    merged = []
    used = [False] * len(boxes)
    for i in range(len(boxes)):
        if used[i]:
            continue
        x1, y1, x2, y2 = boxes[i]["box"]
        merged_box = [x1, y1, x2, y2]
        for j in range(i + 1, len(boxes)):
            if used[j]:
                continue
            x3, y3, x4, y4 = boxes[j]["box"]
            iou = calculate_iou([x1, y1, x2, y2], [x3, y3, x4, y4])
            if iou > iou_thresh:
                merged_box = [
                    min(merged_box[0], x3),
                    min(merged_box[1], y3),
                    max(merged_box[2], x4),
                    max(merged_box[3], y4),
                ]
                used[j] = True
        used[i] = True
        merged.append({"box": merged_box})
    return merged

# 사람의 보트 탑승 여부 판별
def is_person_on_boat(person_box, boat_box, bottom_ratio=0.15):
    px1, py1, px2, py2 = person_box
    bx1, by1, bx2, by2 = boat_box
    foot_x = (px1 + px2) / 2
    foot_y = py2 - (py2 - py1) * bottom_ratio
    return (bx1 <= foot_x <= bx2) and (by1 <= foot_y <= by2)

# 사람 및 보트 탐지
def detect_people_and_boats(img):
    persons, boats = [], []
    results = passenger_model.predict(img, device=device, verbose=False)[0]
    for i, cls_id in enumerate(results.boxes.cls.cpu().numpy().astype(int)):
        label = passenger_model.model.names[cls_id].lower()
        box = results.boxes.xyxy[i].cpu().numpy()
        if label == "person":
            persons.append({"box": box})
        elif label in ("boat", "ship"):
            boats.append({"box": box})
    persons = merge_close_boxes(persons, iou_thresh=0.2)
    boats = merge_close_boxes(boats, iou_thresh=0.5)
    return persons, boats

# 보트별 탑승 인원 매칭 (보트 좌표와 사람 좌표 함께 사용)
def match_people_to_boats(persons, boats):
    for boat in boats:
        boat["persons"] = []
        for person in persons:
            if is_person_on_boat(person["box"], boat["box"]):
                boat["persons"].append(person)
    return boats

# 결과 시각화 및 저장
def visualize_and_save_passenger(img, boats, img_name, output_dir):
    result = img.copy()
    for i, boat in enumerate(boats):
        bx1, by1, bx2, by2 = map(int, boat["box"])
        cv2.rectangle(result, (bx1, by1), (bx2, by2), (0, 255, 0), 2)
        cv2.putText(
            result,
            f"Boat {i+1}: {len(boat['persons'])} persons",
            (bx1, by1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (255, 255, 255),
            2,
        )
        for person in boat["persons"]:
            px1, py1, px2, py2 = map(int, person["box"])
            cv2.rectangle(result, (px1, py1), (px2, py2), (0, 0, 255), 2)
    output_dir.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_dir / img_name), result)

print("[INFO] 승선 인원수 탐지 함수 준비 완료")


In [ ]:
# 승선 인원수 탐지 실행
print("=" * 60)
print("승선 인원수 탐지 시작")
print("=" * 60)

# 승선 인원수 탐지는 보트 유형 분류 결과를 활용
# 보트 유형 분류에서 나온 보트 좌표와 사람 좌표를 함께 사용
# result/boat_classification/ 폴더의 *_result.jpg 파일들을 사용
PASSENGER_INPUT_PATH = BOAT_CLASSIFICATION_RESULT_DIR
if not PASSENGER_INPUT_PATH.exists():
    print(f"[WARN] 보트 유형 분류 결과 폴더가 없습니다: {get_relative_path(PASSENGER_INPUT_PATH)}")
    print(f"[INFO] 대신 원본 테스트 이미지 폴더를 사용합니다: {get_relative_path(TEST_PATH)}")
    image_files_for_passenger = glob.glob(str(TEST_PATH / "*.*"))
else:
    image_files_for_passenger = glob.glob(str(PASSENGER_INPUT_PATH / "*_result.jpg"))
    if not image_files_for_passenger:
        print(f"[WARN] 보트 유형 분류 결과 이미지 파일이 없습니다: {get_relative_path(PASSENGER_INPUT_PATH)}")
        print(f"[INFO] 대신 원본 테스트 이미지 폴더를 사용합니다: {get_relative_path(TEST_PATH)}")
        image_files_for_passenger = glob.glob(str(TEST_PATH / "*.*"))

image_files_for_passenger = [f for f in image_files_for_passenger if os.path.splitext(f)[1].lower() in ['.jpg', '.jpeg', '.png', '.bmp']]
print(f"[INFO] 처리할 이미지: {len(image_files_for_passenger)}개")

for path in sorted(image_files_for_passenger):
    img = cv2.imread(path)
    if img is None:
        print(f"[WARN] 이미지 로드 실패: {path}")
        continue
    
    persons, boats = detect_people_and_boats(img)
    boats = match_people_to_boats(persons, boats)
    # 승선 인원수 탐지 결과를 보트 유형 분류 결과 파일에 덮어쓰기
    # *_result.jpg 파일을 그대로 덮어써서 최종 통합 결과로 만듦
    img_name = os.path.basename(path)  # _result.jpg 그대로 유지
    visualize_and_save_passenger(img, boats, img_name, BOAT_CLASSIFICATION_RESULT_DIR)
    print(f"[INFO] 처리 완료: {img_name} - 보트 {len(boats)}개, 총 탑승 인원 {sum(len(b['persons']) for b in boats)}명")

print(f"[완료] 승선 인원수 탐지 결과 저장: {get_relative_path(BOAT_CLASSIFICATION_RESULT_DIR)}")


## 4. 번호판 OCR (Plate OCR) - 독립 실행


In [ ]:
# 번호판 OCR 실행 (src/plate_ocr_inference.py 사용)
print("=" * 60)
print("번호판 OCR 시작")
print("=" * 60)

# plate_ocr_inference.py 실행
import subprocess
import shutil

result = subprocess.run(
    [sys.executable, str(BASE_DIR / "src" / "plate_ocr_inference.py")],
    cwd=str(BASE_DIR),
    capture_output=True,
    text=True,
    encoding='utf-8',
    errors='replace'
)

print(result.stdout)
if result.stderr:
    print("[WARN] stderr:", result.stderr)

# 결과를 result/plate_ocr/로 복사
ocr_output_dir = BASE_DIR / "runs" / "ocr" / "OCR_FINAL_1029"
if ocr_output_dir.exists():
    # 결과 폴더가 있으면 복사
    if (ocr_output_dir / "ocr_results.csv").exists():
        shutil.copy2(ocr_output_dir / "ocr_results.csv", PLATE_RESULT_DIR / "ocr_results.csv")
    if (ocr_output_dir / "overlay").exists():
        overlay_src = ocr_output_dir / "overlay"
        overlay_dst = PLATE_RESULT_DIR / "overlay"
        if overlay_dst.exists():
            shutil.rmtree(overlay_dst)
        shutil.copytree(overlay_src, overlay_dst)
    if (ocr_output_dir / "crops").exists():
        crops_src = ocr_output_dir / "crops"
        crops_dst = PLATE_RESULT_DIR / "crops"
        if crops_dst.exists():
            shutil.rmtree(crops_dst)
        shutil.copytree(crops_src, crops_dst)
    print(f"[INFO] 결과를 {get_relative_path(PLATE_RESULT_DIR)}로 복사 완료")

CSV_PATH = PLATE_RESULT_DIR / "ocr_results.csv"
OVERLAY_DIR = PLATE_RESULT_DIR / "overlay"
CROP_DIR = PLATE_RESULT_DIR / "crops"

print(f"\n✅ 번호판 OCR 완료!")
print(f"[DONE] 결과 CSV: {get_relative_path(CSV_PATH)}")
print(f"[DONE] 오버레이 폴더: {get_relative_path(OVERLAY_DIR)}")
print(f"[DONE] 크롭 폴더: {get_relative_path(CROP_DIR)}")


## 5. 결과 확인


In [ ]:
# 결과 요약 출력
import pandas as pd
import matplotlib.pyplot as plt

print("=" * 60)
print("결과 요약")
print("=" * 60)

# 보트 유형 분류 및 승선 인원수 탐지 통합 결과
# *_result.jpg 파일들이 최종 통합 결과 (보트 유형 + 탑승 인원 수)
boat_result_files = [f for f in list(BOAT_CLASSIFICATION_RESULT_DIR.glob("*_result.jpg")) + list(BOAT_CLASSIFICATION_RESULT_DIR.glob("*_result.png"))]
print(f"\n[보트 유형 분류 및 승선 인원수 탐지 통합 결과]")
print(f"  결과 이미지: {len(boat_result_files)}개")
print(f"  저장 위치: {get_relative_path(BOAT_CLASSIFICATION_RESULT_DIR)}")

# 번호판 OCR 결과
if CSV_PATH.exists():
    df = pd.read_csv(CSV_PATH)
    print(f"\n[번호판 OCR]")
    print(f"  총 탐지: {len(df)}개")
    print(f"  유효한 번호판: {df['is_valid'].sum()}개")
    print(f"  저장 위치: {get_relative_path(PLATE_RESULT_DIR)}")
    
    if len(df) > 0:
        print(f"\n[OCR 결과 샘플]")
        # CSV의 image 경로도 상대 경로로 변환하여 표시
        df_display = df.copy()
        df_display['image'] = df_display['image'].apply(lambda x: str(get_relative_path(x)) if pd.notna(x) else x)
        print(df_display[['image', 'ocr_text', 'norm_text', 'is_valid']].head(10).to_string(index=False))
else:
    print(f"\n[번호판 OCR]")
    print(f"  결과 파일 없음")

print("\n" + "=" * 60)
